##### Reference raster probably must be the same size (width / height) as the imagery, resolution also

In [2]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.transform import Affine

def check_compatibility(image_path, patch_size):
    """
    Check if the image dimensions are divisible by the patch size.
    
    Args:
        image_path (str): Path to the input image.
        patch_size (int): Size of the patches.
    
    Returns:
        compatible (bool): True if the image dimensions are compatible with the patch size.
    """
    with rasterio.open(image_path) as src:
        width, height = src.width, src.height
        if width % patch_size == 0 and height % patch_size == 0:
            return True
        else:
            return False


def generate_patches(image_path, patch_size, stride, output_dir, offset_left=0, offset_top=0, as_geotif=True):
    """
    Extract patches from a Sentinel-2 image and either save them as GeoTIFF files or return as a NumPy array.
    
    Args:
        image_path (str): Path to the input Sentinel-2 image.
        patch_size (int): Size of the square patches (in pixels).
        stride (int): Step size for sliding the window.
        output_dir (str): Directory to save the extracted patches (if as_geotif is True).
        offset_left (int): Number of pixels to ignore from the left edge of the image.
        offset_top (int): Number of pixels to ignore from the top edge of the image.
        as_geotif (bool): If True, patches are saved as GeoTIFF files; if False, returned as a NumPy array.
    
    Returns:
        patches_array (np.ndarray): Numpy array of shape (num_tiles, bands, patch_size, patch_size) if as_geotif is False.
    """
    if as_geotif:
        os.makedirs(output_dir, exist_ok=True)

    # Open the image and get properties
    with rasterio.open(image_path) as src:
        width, height = src.width, src.height
        transform = src.transform
        crs = src.crs
        dtype = src.dtypes[0]
        num_bands = src.count

        # Adjust width, height, and transform based on offsets
        if offset_left > 0:
            width -= offset_left
            transform = transform * Affine.translation(offset_left, 0)
        if offset_top > 0:
            height -= offset_top
            transform = transform * Affine.translation(0, offset_top)

        # Check if the adjusted image size is compatible with the patch size
        if check_compatibility(image_path, patch_size):
            # If compatible, no cropping necessary
            cropped_image = src.read()
            cropped_transform = transform
            cropped_width, cropped_height = width, height
        else:
            # If not compatible, crop the image to make it tileable
            tileable_width = (width - patch_size) // stride * stride + patch_size
            tileable_height = (height - patch_size) // stride * stride + patch_size
            crop_window = Window(offset_left, offset_top, tileable_width, tileable_height)
            cropped_transform = src.window_transform(crop_window)
            cropped_image = src.read(window=crop_window)
            cropped_width, cropped_height = tileable_width, tileable_height

    # Initialize a list to store patches if as_geotif is False
    patches_list = [] if not as_geotif else None

    # Loop over rows and columns to extract patches
    patch_counter = 0
    for row in range(0, cropped_height - patch_size + 1, stride):
        for col in range(0, cropped_width - patch_size + 1, stride):
            # Compute the translation in pixel space (scaled by pixel size)
            x_offset = col * cropped_transform.a  # Horizontal offset in geospatial units
            y_offset = row * cropped_transform.e  # Vertical offset in geospatial units
            patch_transform = cropped_transform * Affine.translation(x_offset / cropped_transform.a, y_offset / cropped_transform.e)

            # Extract the patch data
            patch_data = cropped_image[:, row:row + patch_size, col:col + patch_size]

            if as_geotif:
                # Save the patch as a GeoTIFF
                patch_filename = f"patch_{str(patch_counter).zfill(8)}.tif"
                patch_path = os.path.join(output_dir, patch_filename)
                with rasterio.open(
                    patch_path,
                    'w',
                    driver='GTiff',
                    height=patch_size,
                    width=patch_size,
                    count=num_bands,
                    dtype=dtype,
                    crs=crs,
                    transform=patch_transform,
                ) as dst:
                    dst.write(patch_data)
            else:
                # Append the patch data to the list
                patches_list.append(patch_data)

            patch_counter += 1

    # If as_geotif is False, convert the patches list to a NumPy array
    if not as_geotif:
        patches_array = np.stack(patches_list, axis=0)
        return patches_array

    return None

#### Tile Imagery

In [3]:
# SETTINGS
image_path = r"C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\composite_subset.tif"
patch_size = 10
stride = 10
offset_left = 2
offset_top = 2
output_dir= None
as_geotif = False

In [4]:
features_array = generate_patches(
    image_path=image_path,
    patch_size = patch_size,
    stride = stride,
    offset_left = offset_left,
    offset_top = offset_top,
    output_dir=output_dir,
    as_geotif=as_geotif
)

print(f"NumPy array shape: {features_array.shape}")

NumPy array shape: (428238, 10, 10, 10)


#### Tile reference raster

In [5]:
# SETTINGS
image_path = r"C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\berlin_lcz_GT_fullres.tif"
patch_size = 10
stride = 10
offset_left = 2
offset_top = 2
output_dir= None
as_geotif = False

In [6]:
labels_array = generate_patches(
    image_path=image_path,
    patch_size = patch_size,
    stride = stride,
    offset_left = offset_left,
    offset_top = offset_top,
    output_dir=output_dir,
    as_geotif=as_geotif
)

print(f"NumPy array shape: {labels_array.shape}")

NumPy array shape: (428238, 1, 10, 10)


#### Central pixel labels extraction

In [13]:
# for each reference tile get the classvalue of the central pixel and create vector of labels
labels_list=[]

for i in range(0, features_array.shape[0]):
    central_pixel_class = int(labels_array[i, :, int(np.ceil(patch_size/2)), int(np.ceil(patch_size/2))][0])
    labels_list.append(central_pixel_class)

central_pixel_label_array = np.array(labels_list)
print(central_pixel_label_array.shape)
print(np.unique(labels_array))

(428238,)
[ 0  2  4  5  6  8  9 11 12 13 14 16 17]


#### Filter tiles with existing label

In [14]:
# SETTING - oznacenie backgroundu, alebo classe 0 alebo nodata...
no_class_value = 0

In [15]:
filtered_input_features = features_array[central_pixel_label_array!=no_class_value]
filtered_labels_array = central_pixel_label_array[central_pixel_label_array!=no_class_value]

print(f"Filtered input_features shape: {filtered_input_features.shape}")
print(f"Filtered labels_array shape: {filtered_labels_array.shape}")

Filtered input_features shape: (24537, 10, 10, 10)
Filtered labels_array shape: (24537,)


#### Saving Numpy arrays

In [16]:
# saving scene - tiles of features
np.save(r'C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\tiled_10x10_stride10_offleft2_offtop2_fullres_features.npy',
        features_array)

In [17]:
# save GT data - tiles of features with label + label vector
np.save(r'C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\tiled_10x10_stride10_offleft2_offtop2_fullres_GT_features.npy',
        filtered_input_features)

np.save(r'C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\tiled_10x10_stride10_offleft2_offtop2_fullres_GT_labels.npy',
        filtered_labels_array)

#### Statistics of GT data

In [18]:
#  number of tiles per class
np.unique(filtered_labels_array, return_counts=True)

(array([ 2,  4,  5,  6,  8,  9, 11, 12, 13, 14, 16, 17]),
 array([1534,  577, 2448, 4010, 1654,  761, 4960, 1028, 1050, 4424,  359,
        1732], dtype=int64))